<a href="https://colab.research.google.com/github/zabbonat/AML_project/blob/master/convertdcmjpg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install pydicom

In [0]:
from querymachine import QueryMachine
import matplotlib.pyplot as plt
import pydicom
import os
import pandas as pd
from tqdm import tqdm_notebook
pd.options.display.max_rows = 4000

In [0]:

qm = QueryMachine()
response = qm.getSeries()

In [0]:
savepath = '/content/drive/My Drive/progetto/DILE'
os.makedirs(savepath, exist_ok=True)
# b_dir = savepath + '/benign'
# m_dir = savepath + '/malignant'
# raw_dir = savepath + '/raw_dcm'
train_dir = savepath + '/train'
valid_dir = savepath + '/valid'
# os.makedirs(b_dir, exist_ok=True)
# os.makedirs(m_dir, exist_ok=True)
# os.makedirs(raw_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)
for pth in [train_dir, valid_dir]:
    os.makedirs(os.path.join(pth, 'benign'), exist_ok=True)
    os.makedirs(os.path.join(pth, 'malignant'), exist_ok=True)
    os.makedirs(os.path.join(pth, 'normal'), exist_ok=True)

In [0]:
calc = pd.read_csv('calc_case_description_train_set.csv')
mass = pd.read_csv('mass_case_description_train_set.csv')
labels_train = calc.append(mass, sort=False)
labels_train['set'] = 'train'
labels_train.set
calc = pd.read_csv('calc_case_description_test_set.csv')
mass = pd.read_csv('mass_case_description_test_set.csv')
labels_test = calc.append(mass, sort=False)
labels_test['set'] = 'test'
print(labels_train.shape, labels_test.shape)
labels = labels_train.append(labels_test)
labels.shape

(2864, 18) (704, 18)


(3568, 18)

In [0]:
labels.index = range(labels.shape[0])

In [0]:

ids = [r['SeriesInstanceUID'] for r in response.json() if 'cropped' in r['SeriesDescription']]

len(ids), len(response.json())

(107, 6775)

In [0]:
labs = {}
for f in ids:
    for row in labels.loc[:, [c for c in labels.columns if 'file path' in c]].itertuples(index=True, name=None):
        for name in row[1:]:
            if f in name.split('/'):
                label = labels.loc[row[0],'pathology'].split('_')[0].lower()
                img_set = labels.loc[row[0], 'set']
                labs[f] = (label, img_set)
len(labs)

107

In [0]:
train_counter = 0
test_counter = 0
for k,v in labs.items():
    if v[1] == 'train':
        train_counter += 1
    else:
        test_counter += 1
train_counter, test_counter

(94, 13)

In [0]:
def downloadImages(response_list, labs, savepath):
    roi_counter = 0
    img_counter = 0
    failed_reads = 0
    for item in tqdm_notebook(response_list):
    
      series_id = item['SeriesInstanceUID']
      if series_id not in labs.keys():
          continue
#         if 'ROI' in item['SeriesDescription']:
#             roi_counter += int(item['ImageCount'])
#         else:
#             img_counter += int(item['ImageCount'])
      label, img_set = labs[series_id]
      qm.getSeriesImages(series_id, savepath=savepath)
      images = [f for f in os.listdir(savepath) if '.dcm' in f]
     
      for f in images:
        
        filepath = os.path.join(savepath, f)
        ds = pydicom.dcmread(filepath)
  #             try:
  #                 ds = pydicom.dcmread(filepath)
  #             except:
  #                 continue
        os.remove(filepath)
  #             if not label:
  #                 return ds
        if img_set == 'train':
            jpg_savepath = savepath + '/train'
            print('sono nel train')
        else:
            jpg_savepath = savepath + '/valid'
            print('sono nel valid!')
        jpg_savepath = jpg_savepath + '/' + label
        jpg_name = series_id + '_' + f
        jpg_filepath = os.path.join(jpg_savepath, jpg_name)
        jpg_filepath = jpg_filepath.replace('.dcm', '.jpg')
        ds.decompress()
        img = ds.pixel_array
        plt.imsave(jpg_filepath, ds.pixel_array,
                  vmin=0, vmax=2**16, format='jpg', cmap='gray')
          

In [0]:

downloadImages(response.json(), labs, savepath)